In [1]:
import io
from google.oauth2 import service_account
from google.cloud import speech

In [2]:
client_file = 'data/keys/google_stt.json'
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)

In [3]:
audio_file = 'data/audio/filename.wav'
with io.open(audio_file, 'rb') as f:
    content = f.read()
audio = speech.RecognitionAudio(content=content)

In [6]:
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US'
)

In [7]:
response = client.recognize(config=config, audio=audio)

results {
  alternatives {
    transcript: "open the door please"
    confidence: 0.967354298
  }
  result_end_time {
    seconds: 2
    nanos: 260000000
  }
  language_code: "en-us"
}
total_billed_time {
  seconds: 3
}
request_id: 7419787476229877833



In [10]:
print(response.results[0].alternatives[0].transcript)

open the door please
